In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyomo
from pyomo.environ import *
import xlwings as xw

In [2]:
#def read_excel(self,filename="VerdeReducido2.xlsx"):
WB=""
def read_excel(filename="VerdeReducido2.xlsx"):
    global WB
    wb = xw.Book(filename)

    try:
        sht = wb.sheets['Resultado']
    except:
        wb.sheets.add("Resultado")

    producto = wb.sheets["Producto"].range('A1').options(pd.DataFrame, expand='table',dtype="float64").value
    contribucion = wb.sheets["Contribucion"].range('A1').options(pd.DataFrame, expand='table').value
    requerimientos = wb.sheets["Balance"].range('A1').options(pd.DataFrame, expand='table').value
    Inventario = wb.sheets["Inventario"].range('A1').options(pd.DataFrame, expand='table').value
    Restricciones = wb.sheets["Restricciones"].range('A1').options(pd.DataFrame, expand='table').value

    data = {
        'Varproducto': producto,
        'contribucion': contribucion,
        'requeri': requerimientos,
        'Inventario': Inventario,
        "Restricciones":Restricciones}
    
    WB=wb
    #self.WB= wb
    return data

In [14]:
Datos = read_excel('./info.xlsx')

In [15]:
Datos["Inventario"][Datos["Inventario"]["MineralSiNo"]>0]

,Kgs,MineralSiNo
Materiales,,
MINERAL JATCO,3000.0,1.0
MINERAL NMEX,1671.0,1.0
MINERAL CUBOS,200.0,1.0
GRANALLA ALULIQ,631.0,1.0
MINERAL CUBOS ALULIQ,300.0,1.0
MINERAL JATCO TEM,187.0,1.0
g2v,600.0,1.0


In [16]:
#def Optimiza(self):
    #Generar un try y catch para ver si ya se cargo los datos o poner otros
    #Datos = read_excel('./VerdeReducido2.xlsx')
    
    #Datos = self.Datos
Datos["Varproducto"].fillna(0,inplace=True)
Datos["contribucion"].fillna(0,inplace=True)
Datos["requeri"]= Datos["requeri"][ Datos["requeri"]["MAX"]!= 0]
#Datos["requeri"] # Eliminando variables
# Variables a optimizar pollo carne arroz etc.
Ingredients =  list(Datos["Varproducto"].index)
#Ingredients
Varproducto = Datos["Varproducto"]
#Restriccion de minerales
RestriMinerales = Datos["Inventario"][Datos["Inventario"]["MineralSiNo"]>0].index

model = ConcreteModel(name="The Verde Problem")

model.ingredient_vars = Var(Ingredients, bounds=(0,None), doc="The amount of each ingredient that is used")

model.obj = Objective(expr=sum(Varproducto.Precio[i]*model.ingredient_vars[i] for i in Ingredients)/sum(model.ingredient_vars[i]*Datos["contribucion"]["Eficiencia"][i]  for i in Ingredients) ,sense=minimize, doc="Total Cost of Ingredients per can")

#Restriccion de kg finales entonces depende de la eficiencia
# Hay que modificar.

model.c0 = Constraint(expr=sum(model.ingredient_vars[i]*Datos["contribucion"]["Eficiencia"][i]  for i in Ingredients) <= Datos["Restricciones"].loc["Kg"][1], doc="PercentagesSum")
model.c02 = Constraint(expr=sum(model.ingredient_vars[i]*Datos["contribucion"]["Eficiencia"][i]  for i in Ingredients) >= Datos["Restricciones"].loc["Kg"][0], doc="PercentagesSum2")

### Restriccion dekg para los minerales sean menores a 1400
#Con la eficiencia se calcula el kg
if len(RestriMinerales)>0:
    #model.c03 = Constraint(expr=sum(model.ingredient_vars[i]*Datos["contribucion"]["Eficiencia"][i]  for i in RestriMinerales) <= Datos["Restricciones"].loc["Mineral"][1], doc="PercentagesSum3")
    #model.c04 = Constraint(expr=sum(model.ingredient_vars[i]*Datos["contribucion"]["Eficiencia"][i]  for i in RestriMinerales) >= Datos["Restricciones"].loc["Mineral"][0], doc="PercentagesSum4")
    model.c03 = Constraint(expr=sum(model.ingredient_vars[i]  for i in RestriMinerales) <= Datos["Restricciones"].loc["Mineral"][1], doc="PercentagesSum3")
    model.c04 = Constraint(expr=sum(model.ingredient_vars[i]  for i in RestriMinerales) >= Datos["Restricciones"].loc["Mineral"][0], doc="PercentagesSum4")


composicionesQ = list(Datos["requeri"].index)

#you can use rules to construct the constraints for all p \in P.

def constraint1_ruleMin(m,p):
#    return sum(Datos["contribucion"][p][i] * model.ingredient_vars[i] for i in Ingredients) >= Datos["requeri"]["MIN"][p]
    return ( (sum(Datos["contribucion"][p][i] * model.ingredient_vars[i]*Datos["contribucion"]["Eficiencia"][i] for i in Ingredients)/
                      (sum( model.ingredient_vars[i]*Datos["contribucion"]["Eficiencia"][i] for i in Ingredients)) )/100
                      >= Datos["requeri"]["MIN"][p])

#model.c1 = Constraint([p for p in Ps],rule=constraint2_rule)

def constraint1_ruleMax(m,p):
#   return sum(Datos["contribucion"][p][i] * model.ingredient_vars[i] for i in Ingredients) <= Datos["requeri"]["MAX"][p]
    return ( (sum(Datos["contribucion"][p][i] * model.ingredient_vars[i]*Datos["contribucion"]["Eficiencia"][i] for i in Ingredients)/
                      (sum( model.ingredient_vars[i]*Datos["contribucion"]["Eficiencia"][i] for i in Ingredients)) )/100
                      <= Datos["requeri"]["MAX"][p])


def constraint1_ruleInventario(m,p):
    return (model.ingredient_vars[p]  <= Datos["Inventario"]["Kgs"][p])

model.c1 = Constraint(composicionesQ,rule=constraint1_ruleMin)

model.c2 = Constraint(composicionesQ,rule=constraint1_ruleMax)

model.c3 = Constraint(Ingredients,rule=constraint1_ruleInventario)

solver = SolverFactory('ipopt')
#status = solver.solve(model,tee=True)
status = solver.solve(model)


In [17]:
print("Status = %s" % status.solver.termination_condition)

for i in Ingredients:
    print("%s = %f" % (model.ingredient_vars[i], value(model.ingredient_vars[i])))

print("Objective = %f" % value(model.obj))

Status = optimal
ingredient_vars[LAMINA LITOGRAFICA] = 0.000001
ingredient_vars[HOJUELA ALUMINIO] = 0.000003
ingredient_vars[SILICIO ] = 0.000013
ingredient_vars[ALUMINIO GRUESO] = 0.000003
ingredient_vars[RIN CHICO] = 0.000001
ingredient_vars[PACA PREMIUM FOREMEX] = 0.000004
ingredient_vars[RAD. PREMIUM ] = 0.000002
ingredient_vars[RAD.ALUMINIO 2da. ] = 0.000003
ingredient_vars[RAD. VENA DE COBRE] = 0.000001
ingredient_vars[PERFIL VENTANERIA] = 0.000002
ingredient_vars[PERFIL GRUESO] = 0.000003
ingredient_vars[PERFIL TROQUELADP CCDDC] = 0.000059
ingredient_vars[PERFIL TROQUELADO LT18] = 225.273921
ingredient_vars[PERFIL TROQUELADO 8N] = 497.734469
ingredient_vars[CABEZA DE ALUMINIO] = 0.000004
ingredient_vars[MULTIPLE ADMISION] = 0.000004
ingredient_vars[PISTON] = 0.000004
ingredient_vars[CABLE PURO] = 0.000001
ingredient_vars[CABLE ALCu] = 11.822535
ingredient_vars[PACA DELGADO] = 0.000003
ingredient_vars[P. DELGADO HIGH COMODITY] = 0.000003
ingredient_vars[DELGADO A GRANEL] = 0.0000

In [18]:
sht = WB.sheets["Balance"]
sht.range('B20').value= (p,Datos["requeri"]["MIN"][p] ,tmp, Datos["requeri"]["MAX"][p] )

In [19]:
indx=20
for p in composicionesQ:
    tmp =(sum(Datos["contribucion"][p][i] * value(model.ingredient_vars[i])*Datos["contribucion"]["Eficiencia"][i] for i in Ingredients)/
                      (sum( value(model.ingredient_vars[i])*Datos["contribucion"]["Eficiencia"][i] for i in Ingredients)  ) )/100
    print("%s -> %f <= %f<= %f" % (p,Datos["requeri"]["MIN"][p] ,tmp, Datos["requeri"]["MAX"][p] ) )    
    sht.range('B'+str(indx)).value= (p,Datos["requeri"]["MIN"][p] ,tmp, Datos["requeri"]["MAX"][p] )
    indx+=1
#    print ("%s -> %f <= %f<= %f" % (p,Datos["requeri"]["MIN"][p] ,sum(Datos["contribucion"][p][i] * value(model.ingredient_vars[i]) for i in Ingredients), Datos["requeri"]["MAX"][p] ) )

SI -> 0.100000 <= 0.100000<= 0.118000
CU -> 0.018000 <= 0.018000<= 0.033000
FE -> 0.006500 <= 0.010000<= 0.010000
MG -> 0.001200 <= 0.003000<= 0.003000
ZN -> 0.000000 <= 0.010613<= 0.028000
MN -> 0.000000 <= 0.002629<= 0.005000
NI -> 0.000000 <= 0.003053<= 0.005000
Sn -> 0.000000 <= 0.000152<= 0.003000


In [20]:

ing = []
canIng = []
ing.append("Status = ")
canIng.append(str(status.solver.termination_condition) )
for i in Ingredients:
    ing.append(str(model.ingredient_vars[i]).replace("ingredient_vars[","").replace("]",""))
    canIng.append( round( float(value(model.ingredient_vars[i])) ,4) )

ing.append("Objective = ")
canIng.append(str(value(model.obj)))

#self.DataFrame_Resultado = pd.DataFrame(canIng,ing,columns=["Cantidad"])
DataFrame_Resultado = pd.DataFrame(canIng,ing,columns=["Cantidad"])


sht = WB.sheets["Resultado"]
sht.range('A1').value = DataFrame_Resultado

In [10]:
def GuardarDatos(self):
    #sht = self.WB.sheets["Resultado"]
    sht = WB.sheets["Resultado"]
    sht.range('A1').value = self.DataFrame_Resultado

In [22]:
! start .